In [8]:
import numpy as np
import pandas as pd

In [39]:
def sliding_window_preprocessing(sequences, history_window, horizon=1, target_col=None):
    if target_col:
        target = sequences.pop(target_col)
        sequences[target_col] = target
    sequences = np.array(sequences)
    
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + history_window
        horizon_end_ix = end_ix + horizon - 1
        # check if we are beyond the sequence
        if horizon_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:horizon_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

arr = np.array([[ 10, 15, 25], [ 20, 25, 45], [ 30, 35, 65],
                [ 40, 45, 85], [ 50, 55, 105], [ 60, 65, 125],
                [ 70, 75, 145], [ 80, 85, 165], [ 90, 95, 185]])
df = pd.DataFrame(arr, columns=["num1", "num2", "sum"])

X, y = sliding_window_preprocessing(df, 2, 2)
for i in range(len(X)):
    print(X[i], y[i])

[[10 15]
 [20 25]] [45 65]
[[20 25]
 [30 35]] [65 85]
[[30 35]
 [40 45]] [ 85 105]
[[40 45]
 [50 55]] [105 125]
[[50 55]
 [60 65]] [125 145]
[[60 65]
 [70 75]] [145 165]
[[70 75]
 [80 85]] [165 185]


In [ ]:
def sliding_window_with_breakdown(sequence, history_window, horizon, breakdown_indices):
    pass

In [2]:
def feature_scaling(sequence, method):
    pass

def reverse_scaling(sequence):
    pass